In [2]:
#ollama
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from numba import cuda,jit
import cupy as cp
import bs4
import chromadb
from langchain.callbacks.base import BaseCallbackHandler
from langchain.schema import LLMResult
import sys
import time
import json

In [3]:
class StreamingStdOutCallbackHandler(BaseCallbackHandler):
    def on_llm_new_token(self, token: str, **kwargs) -> None:
        sys.stdout.write(token)
        sys.stdout.flush()
        time.sleep(0.01)  # Add a small delay to simulate typing

    def on_llm_end(self, response: LLMResult, **kwargs) -> None:
        sys.stdout.write("\n")

In [4]:
model_local = ChatOllama(model='llama3')
#client1 = chromadb.PersistentClient(path="./vectorstore1")

In [4]:
# Training
def load_document(path): 
    document_loader = PyPDFDirectoryLoader(path)
    return document_loader.load()
client = chromadb.Client()

In [5]:
#collection = client.create_collection(name="rag-chroma1")

In [6]:
# t=Training
path= 'data'
document = load_document(path)
document[1]

Document(metadata={'source': 'data\\7434402.pdf', 'page': 1}, page_content='IUIUIUIUIUIUIUIUIUIUIUIUIUIUIU!UIUIUIUIUIUIOlO!CJT\nTheGanges\ninMythandHistory')

In [6]:
# Training
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=7500, chunk_overlap=100)
doc_split = text_splitter.split_documents(document)

In [7]:
# Training
with cp.cuda.Device(0):
    vectorstore = Chroma.from_documents(
        documents=doc_split,
        embedding=OllamaEmbeddings(model='nomic-embed-text'),
        persist_directory="./chroma_db"
    )

In [8]:
# Training
vectorstore.persist()

c:\Users\KIRAN\Documents\ML\Dhara\venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [5]:
streaming_callback = StreamingStdOutCallbackHandler()
embeddings = OllamaEmbeddings(model='nomic-embed-text')
loaded_vectorstore = Chroma(persist_directory="./chroma_db1", embedding_function=embeddings)
loaded_vectorstore.get()

{'ids': [],
 'embeddings': None,
 'metadatas': [],
 'documents': [],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents']}

In [6]:
#retriever = vectorstore.as_retriever()
retriever = loaded_vectorstore.as_retriever()

In [5]:
'''before_rag_template = "{topic}"
print("Before using RAG:")
before_rag_prompt = ChatPromptTemplate.from_template(before_rag_template)
before_rag_chain = before_rag_prompt | model_local | StrOutputParser()
print(before_rag_chain.invoke({"topic":"What is proposed in the paper Attention is All You Need ?"}))'''

'before_rag_template = "{topic}"\nprint("Before using RAG:")\nbefore_rag_prompt = ChatPromptTemplate.from_template(before_rag_template)\nbefore_rag_chain = before_rag_prompt | model_local | StrOutputParser()\nprint(before_rag_chain.invoke({"topic":"What is proposed in the paper Attention is All You Need ?"}))'

In [7]:
after_rag_template = """answer the question based on the following context:
{context}
Question:{question}
"""
after_rag_prompt = ChatPromptTemplate.from_template(after_rag_template)
after_rag_chain = (
    {"context":retriever, "question":RunnablePassthrough()}
     | after_rag_prompt
     | model_local
     | StrOutputParser()
)

In [8]:
print(after_rag_chain.invoke("reasons behind the pollution of ganga"))


TypeError: 'callbacks' is an invalid keyword argument for print()